In [ ]:
#Scenario
#1. test normalisasi suatu data, apakah distribusinya normal atau tidak-> menggunakn Shapiro-Wilk Test
#2. Kalau normal bisa menggunakan t-test, kalau tidak menggunakn Wilcoxon Effect Size 
#3. Nah keduanya harus ada sample minim 3-5x percobaan untuk membuktikan bahwa nilai akurasinya itu stabil
#4. Inisiatif nya bisa dibuat k-fold cross validation dengan membagi data  menjadi 5 bagian dengan iterasi terakhir sebagai testing
#5. Hasilnya akan ada 5 nilai akurasi di setiap modelnya yang mana akan dibandingkan antar model satu dengan model lain


Data: Using Undersampling Dataset

In [1]:
#import package and library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
from sklearn.model_selection import StratifiedKFold
datasdp = pd.read_csv('D:\AAAAKK SKRIPSWEETT\Bismillah Code Skripsi V2\Dataset\dataset_sdp_undersampling.csv', delimiter=";")

X = datasdp.drop(columns=['bugs'])
y = datasdp['bugs']

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_base, X_train_meta, y_train_base, y_train_meta = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

counts_training = y_train.value_counts()
print("data distribiution data training: ", counts_training)
counts_testing = y_train_base.value_counts()
print("data distribiution data testing: ", counts_testing)

#insialisasi k-fold validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    print(f"Fold {fold} - Data Training Undersampled: {y_train.value_counts().to_dict()}")
    print(f"Fold {fold} - Data Validation: {y_val_fold.value_counts().to_dict()}")

data distribiution data training:  bugs
0    593
1    458
Name: count, dtype: int64
data distribiution data testing:  bugs
0    286
1    239
Name: count, dtype: int64
Fold 1 - Data Training Undersampled: {0: 593, 1: 458}
Fold 1 - Data Validation: {0: 58, 1: 47}
Fold 2 - Data Training Undersampled: {0: 593, 1: 458}
Fold 2 - Data Validation: {0: 57, 1: 48}
Fold 3 - Data Training Undersampled: {0: 593, 1: 458}
Fold 3 - Data Validation: {0: 57, 1: 48}
Fold 4 - Data Training Undersampled: {0: 593, 1: 458}
Fold 4 - Data Validation: {0: 57, 1: 48}
Fold 5 - Data Training Undersampled: {0: 593, 1: 458}
Fold 5 - Data Validation: {0: 57, 1: 48}


In [ ]:
import time
import tracemalloc

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score
from scipy.stats import shapiro, ttest_rel, wilcoxon
import numpy as np

# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

# Inisialisasi model
randomforest_classifier = RandomForestClassifier(
    n_estimators=40,
    max_depth=4,
    min_samples_leaf=5,
    criterion='gini'
)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    randomforest_classifier.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = randomforest_classifier.predict(X_val_fold)

    # Hitung metrik evaluasi    
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

# Akhiri tracking memori dan waktu
current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

# Output
print(f"Waktu pelatihan: {end_time - start_time:.4f} detik")
print(f"Penggunaan memori: {current / 10**6:.4f} MB (saat ini), {peak / 10**6:.4f} MB (puncak)")

# Stop tracing
tracemalloc.stop()

Training fold 1...
Fold 1 - Recall: 0.7447
Fold 1 - F1 Score: 0.7955
Fold 1 - Precision: 0.8537
Fold 1 - ROC AUC Score: 0.8206
Fold 1 - Accuracy: 0.8286
Training fold 2...
Fold 2 - Recall: 0.6875
Fold 2 - F1 Score: 0.7500
Fold 2 - Precision: 0.8250
Fold 2 - ROC AUC Score: 0.7823
Fold 2 - Accuracy: 0.7905
Training fold 3...
Fold 3 - Recall: 0.6667
Fold 3 - F1 Score: 0.7619
Fold 3 - Precision: 0.8889
Fold 3 - ROC AUC Score: 0.7982
Fold 3 - Accuracy: 0.8095
Training fold 4...
Fold 4 - Recall: 0.7083
Fold 4 - F1 Score: 0.7556
Fold 4 - Precision: 0.8095
Fold 4 - ROC AUC Score: 0.7840
Fold 4 - Accuracy: 0.7905
Training fold 5...
Fold 5 - Recall: 0.8333
Fold 5 - F1 Score: 0.8511
Fold 5 - Precision: 0.8696
Fold 5 - ROC AUC Score: 0.8640
Fold 5 - Accuracy: 0.8667
Waktu pelatihan: 4.6090 detik
Penggunaan memori: 0.1907 MB (saat ini), 0.2646 MB (puncak)


In [42]:
#2. extra trees
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score

# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

et_model = ExtraTreesClassifier(
    n_estimators=30,
    max_depth=8,
    min_samples_leaf=4,
    criterion='gini'
)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    et_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = et_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

# Akhiri tracking memori dan waktu
current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

# Output
print(f"Waktu pelatihan: {end_time - start_time:.4f} detik")
print(f"Penggunaan memori: {current / 10**6:.4f} MB (saat ini), {peak / 10**6:.4f} MB (puncak)")

# Stop tracing
tracemalloc.stop()



Training fold 1...
Fold 1 - Recall: 0.6809
Fold 1 - F1 Score: 0.7273
Fold 1 - Precision: 0.7805
Fold 1 - ROC AUC Score: 0.7628
Fold 1 - Accuracy: 0.7714
Training fold 2...
Fold 2 - Recall: 0.6458
Fold 2 - F1 Score: 0.6966
Fold 2 - Precision: 0.7561
Fold 2 - ROC AUC Score: 0.7352
Fold 2 - Accuracy: 0.7429
Training fold 3...
Fold 3 - Recall: 0.6667
Fold 3 - F1 Score: 0.7619
Fold 3 - Precision: 0.8889
Fold 3 - ROC AUC Score: 0.7982
Fold 3 - Accuracy: 0.8095
Training fold 4...
Fold 4 - Recall: 0.7083
Fold 4 - F1 Score: 0.7391
Fold 4 - Precision: 0.7727
Fold 4 - ROC AUC Score: 0.7664
Fold 4 - Accuracy: 0.7714
Training fold 5...
Fold 5 - Recall: 0.7917
Fold 5 - F1 Score: 0.8352
Fold 5 - Precision: 0.8837
Fold 5 - ROC AUC Score: 0.8520
Fold 5 - Accuracy: 0.8571
Waktu pelatihan: 1.9255 detik
Penggunaan memori: 0.1682 MB (saat ini), 0.2390 MB (puncak)


In [43]:
#3. Adapative Boosting (AdaaBoost) using smote technique

from sklearn.ensemble  import  AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

adaboost_model = AdaBoostClassifier(
    n_estimators=50,
    learning_rate=0.1
)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    adaboost_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = adaboost_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

# Akhiri tracking memori dan waktu
current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

# Output
print(f"Waktu pelatihan: {end_time - start_time:.4f} detik")
print(f"Penggunaan memori: {current / 10**6:.4f} MB (saat ini), {peak / 10**6:.4f} MB (puncak)")

# Stop tracing
tracemalloc.stop()



Training fold 1...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Fold 1 - Recall: 0.7447
Fold 1 - F1 Score: 0.7865
Fold 1 - Precision: 0.8333
Fold 1 - ROC AUC Score: 0.8120
Fold 1 - Accuracy: 0.8190
Training fold 2...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Fold 2 - Recall: 0.6250
Fold 2 - F1 Score: 0.7143
Fold 2 - Precision: 0.8333
Fold 2 - ROC AUC Score: 0.7599
Fold 2 - Accuracy: 0.7714
Training fold 3...


...

Fold 3 - Recall: 0.6458
Fold 3 - F1 Score: 0.7294
Fold 3 - Precision: 0.8378
Fold 3 - ROC AUC Score: 0.7703
Fold 3 - Accuracy: 0.7810
Training fold 4...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Fold 4 - Recall: 0.7292
Fold 4 - F1 Score: 0.7865
Fold 4 - Precision: 0.8537
Fold 4 - ROC AUC Score: 0.8120
Fold 4 - Accuracy: 0.8190
Training fold 5...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Fold 5 - Recall: 0.8125
Fold 5 - F1 Score: 0.8478
Fold 5 - Precision: 0.8864
Fold 5 - ROC AUC Score: 0.8624
Fold 5 - Accuracy: 0.8667
Waktu pelatihan: 4.1646 detik
Penggunaan memori: 0.1600 MB (saat ini), 0.2438 MB (puncak)


In [44]:
#4. Gradient Boosting 

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score, classification_report

# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

gb_model = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=0.01,
    min_samples_leaf=5,
    max_depth=3,
    loss='exponential'
)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    gb_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = gb_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

# Akhiri tracking memori dan waktu
current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

# Output
print(f"Waktu pelatihan: {end_time - start_time:.4f} detik")
print(f"Penggunaan memori: {current / 10**6:.4f} MB (saat ini), {peak / 10**6:.4f} MB (puncak)")

# Stop tracing
tracemalloc.stop()


Training fold 1...
Fold 1 - Recall: 0.7021
Fold 1 - F1 Score: 0.7765
Fold 1 - Precision: 0.8684
Fold 1 - ROC AUC Score: 0.8080
Fold 1 - Accuracy: 0.8190
Training fold 2...
Fold 2 - Recall: 0.5417
Fold 2 - F1 Score: 0.6420
Fold 2 - Precision: 0.7879
Fold 2 - ROC AUC Score: 0.7094
Fold 2 - Accuracy: 0.7238
Training fold 3...
Fold 3 - Recall: 0.6250
Fold 3 - F1 Score: 0.7143
Fold 3 - Precision: 0.8333
Fold 3 - ROC AUC Score: 0.7599
Fold 3 - Accuracy: 0.7714
Training fold 4...
Fold 4 - Recall: 0.7083
Fold 4 - F1 Score: 0.7391
Fold 4 - Precision: 0.7727
Fold 4 - ROC AUC Score: 0.7664
Fold 4 - Accuracy: 0.7714
Training fold 5...
Fold 5 - Recall: 0.7917
Fold 5 - F1 Score: 0.8444
Fold 5 - Precision: 0.9048
Fold 5 - ROC AUC Score: 0.8607
Fold 5 - Accuracy: 0.8667
Waktu pelatihan: 4.7648 detik
Penggunaan memori: 0.1668 MB (saat ini), 0.2509 MB (puncak)


In [45]:
#5. Histogram Gradient Boosting

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score, classification_report

# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

hgb_model = HistGradientBoostingClassifier(
    max_iter=50,
    learning_rate=0.01,
    min_samples_leaf=5,
    max_depth=3)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    hgb_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = hgb_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

# Akhiri tracking memori dan waktu
current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

# Output
print(f"Waktu pelatihan: {end_time - start_time:.4f} detik")
print(f"Penggunaan memori: {current / 10**6:.4f} MB (saat ini), {peak / 10**6:.4f} MB (puncak)")

# Stop tracing
tracemalloc.stop()

Training fold 1...
Fold 1 - Recall: 0.6383
Fold 1 - F1 Score: 0.7229
Fold 1 - Precision: 0.8333
Fold 1 - ROC AUC Score: 0.7674
Fold 1 - Accuracy: 0.7810
Training fold 2...
Fold 2 - Recall: 0.5417
Fold 2 - F1 Score: 0.6420
Fold 2 - Precision: 0.7879
Fold 2 - ROC AUC Score: 0.7094
Fold 2 - Accuracy: 0.7238
Training fold 3...
Fold 3 - Recall: 0.6042
Fold 3 - F1 Score: 0.7436
Fold 3 - Precision: 0.9667
Fold 3 - ROC AUC Score: 0.7933
Fold 3 - Accuracy: 0.8095
Training fold 4...
Fold 4 - Recall: 0.7083
Fold 4 - F1 Score: 0.7391
Fold 4 - Precision: 0.7727
Fold 4 - ROC AUC Score: 0.7664
Fold 4 - Accuracy: 0.7714
Training fold 5...
Fold 5 - Recall: 0.8125
Fold 5 - F1 Score: 0.8667
Fold 5 - Precision: 0.9286
Fold 5 - ROC AUC Score: 0.8799
Fold 5 - Accuracy: 0.8857
Waktu pelatihan: 2.1940 detik
Penggunaan memori: 0.2333 MB (saat ini), 0.7746 MB (puncak)


In [49]:
#6. XGBoost 

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score, classification_report

# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

xgb_model = XGBClassifier(
    n_estimators=50, 
    max_depth=4,
    learning_rate=0.01)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    xgb_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = xgb_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')


# Akhiri tracking memori dan waktu
current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

# Output
print(f"Waktu pelatihan: {end_time - start_time:.4f} detik")
print(f"Penggunaan memori: {current / 10**6:.4f} MB (saat ini), {peak / 10**6:.4f} MB (puncak)")

# Stop tracing
tracemalloc.stop()

Training fold 1...
Fold 1 - Recall: 0.6809
Fold 1 - F1 Score: 0.7529
Fold 1 - Precision: 0.8421
Fold 1 - ROC AUC Score: 0.7887
Fold 1 - Accuracy: 0.8000
Training fold 2...
Fold 2 - Recall: 0.5417
Fold 2 - F1 Score: 0.6420
Fold 2 - Precision: 0.7879
Fold 2 - ROC AUC Score: 0.7094
Fold 2 - Accuracy: 0.7238
Training fold 3...
Fold 3 - Recall: 0.6250
Fold 3 - F1 Score: 0.7500
Fold 3 - Precision: 0.9375
Fold 3 - ROC AUC Score: 0.7950
Fold 3 - Accuracy: 0.8095
Training fold 4...
Fold 4 - Recall: 0.7292
Fold 4 - F1 Score: 0.8235
Fold 4 - Precision: 0.9459
Fold 4 - ROC AUC Score: 0.8470
Fold 4 - Accuracy: 0.8571
Training fold 5...
Fold 5 - Recall: 0.8125
Fold 5 - F1 Score: 0.8571
Fold 5 - Precision: 0.9070
Fold 5 - ROC AUC Score: 0.8712
Fold 5 - Accuracy: 0.8762
Waktu pelatihan: 0.7337 detik
Penggunaan memori: 0.1928 MB (saat ini), 0.3723 MB (puncak)


In [ ]:
#7. Categorical Boosting (CatBoost)

from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score, classification_report


# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

cb_model = CatBoostClassifier(
    n_estimators=50,
    loss_function='Logloss',
    learning_rate=0.01,
    depth=5,
    min_data_in_leaf=1,
    random_seed=1,
    logging_level='Silent')

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    cb_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = cb_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')


# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

# Akhiri tracking memori dan waktu
current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

# Output
print(f"Waktu pelatihan: {end_time - start_time:.4f} detik")
print(f"Penggunaan memori: {current / 10**6:.4f} MB (saat ini), {peak / 10**6:.4f} MB (puncak)")

# Stop tracing
tracemalloc.stop()

Training fold 1...
Fold 1 - Recall: 0.6596
Fold 1 - F1 Score: 0.7561
Fold 1 - Precision: 0.8857
Fold 1 - ROC AUC Score: 0.7953
Fold 1 - Accuracy: 0.8095
Training fold 2...
Fold 2 - Recall: 0.6042
Fold 2 - F1 Score: 0.7073
Fold 2 - Precision: 0.8529
Fold 2 - ROC AUC Score: 0.7582
Fold 2 - Accuracy: 0.7714
Training fold 3...
Fold 3 - Recall: 0.6042
Fold 3 - F1 Score: 0.7160
Fold 3 - Precision: 0.8788
Fold 3 - ROC AUC Score: 0.7670
Fold 3 - Accuracy: 0.7810
Training fold 4...
Fold 4 - Recall: 0.6250
Fold 4 - F1 Score: 0.7143
Fold 4 - Precision: 0.8333
Fold 4 - ROC AUC Score: 0.7599
Fold 4 - Accuracy: 0.7714
Training fold 5...
Fold 5 - Recall: 0.7917
Fold 5 - F1 Score: 0.8444
Fold 5 - Precision: 0.9048
Fold 5 - ROC AUC Score: 0.8607
Fold 5 - Accuracy: 0.8667
Waktu pelatihan: 1.1381 detik
Penggunaan memori: 0.1902 MB (saat ini), 0.2746 MB (puncak)


In [69]:
#create stacking ensemble
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


# Mulai tracking waktu dan memori
start_time = time.time()
tracemalloc.start()

estimators_base_model = [
    ('randomforest', randomforest_classifier),
    ('extratrees', et_model),
    ('adaboost', adaboost_model),
    ('gradboost', gb_model),
    ('hgboost', hgb_model),
    ('xgboost', xgb_model),
    ('catboost', cb_model), 
]

#stacked model with  base model and logistic
stack_model = StackingClassifier(
    estimators=estimators_base_model, 
    final_estimator=RandomForestClassifier(), 
    passthrough=False,
    n_jobs=-1)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    stack_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = stack_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

# Akhiri tracking memori dan waktu
current, peak = tracemalloc.get_traced_memory()
end_time = time.time()

# Output
print(f"Waktu pelatihan: {end_time - start_time:.4f} detik")
print(f"Penggunaan memori: {current / 10**6:.4f} MB (saat ini), {peak / 10**6:.4f} MB (puncak)")

# Stop tracing
tracemalloc.stop()

Training fold 1...
Fold 1 - Recall: 0.8298
Fold 1 - F1 Score: 0.8298
Fold 1 - Precision: 0.8298
Fold 1 - ROC AUC Score: 0.8459
Fold 1 - Accuracy: 0.8476
Training fold 2...
Fold 2 - Recall: 0.7292
Fold 2 - F1 Score: 0.7609
Fold 2 - Precision: 0.7955
Fold 2 - ROC AUC Score: 0.7856
Fold 2 - Accuracy: 0.7905
Training fold 3...
Fold 3 - Recall: 0.7292
Fold 3 - F1 Score: 0.8235
Fold 3 - Precision: 0.9459
Fold 3 - ROC AUC Score: 0.8470
Fold 3 - Accuracy: 0.8571
Training fold 4...
Fold 4 - Recall: 0.8333
Fold 4 - F1 Score: 0.8247
Fold 4 - Precision: 0.8163
Fold 4 - ROC AUC Score: 0.8377
Fold 4 - Accuracy: 0.8381
Training fold 5...
Fold 5 - Recall: 0.8750
Fold 5 - F1 Score: 0.8936
Fold 5 - Precision: 0.9130
Fold 5 - ROC AUC Score: 0.9024
Fold 5 - Accuracy: 0.9048
Waktu pelatihan: 47.3980 detik
Penggunaan memori: 1.0528 MB (saat ini), 2.1428 MB (puncak)


Normality testing using Shapiro Wilk and T-test

In [74]:
import numpy as np
from scipy.stats import shapiro, ttest_rel, wilcoxon

# Data hasil K-Fold
model_RF = np.array([0.8206, 0.7823, 0.7982, 0.7840, 0.8640])
model_ET = np.array([0.7628, 0.7352, 0.7982, 0.7664, 0.8520])
model_ADA = np.array([0.8120, 0.7599, 0.7703, 0.8120, 0.8624])
model_GB = np.array([0.8080, 0.7094, 0.7599, 0.7664, 0.8607])
model_HGB = np.array([0.7674, 0.7094, 0.7933, 0.7664, 0.8799])
model_XGB = np.array([0.7887, 0.7094, 0.7950, 0.8470, 0.8712])
model_CB = np.array([0.7953, 0.7582, 0.7670, 0.7599, 0.8607])
model_STA = np.array([0.8459, 0.7856, 0.8470, 0.8377, 0.9024])

# List model untuk perbandingan
models = {
    "RF": model_RF,
    "ET": model_ET,
    "ADA": model_ADA,
    "GB": model_GB,
    "HGB": model_HGB,
    "XGB": model_XGB,
    "CB": model_CB,
    "STA": model_STA
}

# Shapiro-Wilk Test untuk normalitas
print("\n=== Shapiro-Wilk Normality Test ===")
for name, scores in models.items():
    stat, p = shapiro(scores)
    print(f"{name}: W={stat:.4f}, p-value={p:.4f} ({'Normal' if p > 0.05 else 'Non-normal'})")

# Uji T-Test & Wilcoxon antara RF dan model lain
print("\n=== Paired T-Test & Wilcoxon Test Against Stakcing ===")
for name, scores in models.items():
    if name == "STA":
        continue  # Skip diri sendiri
    
    # Uji normalitas kedua model
    _, p_rf = shapiro(model_STA)
    _, p_model = shapiro(scores)

    if p_rf > 0.05 and p_model > 0.05:
        # Jika normal, gunakan T-Test
        t_stat, p_ttest = ttest_rel(model_STA, scores)
        print(f"T-Test STA vs {name}: t-stat={t_stat:.4f}, p-value={p_ttest:.4f} ({'Significant' if p_ttest < 0.05 else 'Not Significant'})")
    else:
        # Jika tidak normal, gunakan Wilcoxon
        w_stat, p_wilcoxon = wilcoxon(model_RF, scores)
        print(f"Wilcoxon STA vs {name}: W={w_stat:.4f}, p-value={p_wilcoxon:.4f} ({'Significant' if p_wilcoxon < 0.05 else 'Not Significant'})")



=== Shapiro-Wilk Normality Test ===
RF: W=0.8645, p-value=0.2449 (Normal)
ET: W=0.9329, p-value=0.6160 (Normal)
ADA: W=0.9240, p-value=0.5559 (Normal)
GB: W=0.9779, p-value=0.9232 (Normal)
HGB: W=0.9303, p-value=0.5983 (Normal)
XGB: W=0.9472, p-value=0.7169 (Normal)
CB: W=0.7883, p-value=0.0648 (Normal)
STA: W=0.9283, p-value=0.5845 (Normal)

=== Paired T-Test & Wilcoxon Test Against Stakcing ===
T-Test STA vs RF: t-stat=3.7384, p-value=0.0201 (Significant)
T-Test STA vs ET: t-stat=8.7400, p-value=0.0009 (Significant)
T-Test STA vs ADA: t-stat=4.2677, p-value=0.0130 (Significant)
T-Test STA vs GB: t-stat=6.4345, p-value=0.0030 (Significant)
T-Test STA vs HGB: t-stat=5.7926, p-value=0.0044 (Significant)
T-Test STA vs XGB: t-stat=2.8450, p-value=0.0466 (Significant)
T-Test STA vs CB: t-stat=5.4142, p-value=0.0056 (Significant)


In [72]:
import numpy as np
from scipy.stats import friedmanchisquare

# Data hasil K-Fold Cross Validation
# Data hasil K-Fold
model_RF = np.array([0.8206, 0.7823, 0.7982, 0.7840, 0.8640])
model_ET = np.array([0.7628, 0.7352, 0.7982, 0.7664, 0.8520])
model_ADA = np.array([0.8120, 0.7599, 0.7703, 0.8120, 0.8624])
model_GB = np.array([0.8080, 0.7094, 0.7599, 0.7664, 0.8607])
model_HGB = np.array([0.7674, 0.7094, 0.7933, 0.7664, 0.8799])
model_XGB = np.array([0.7887, 0.7094, 0.7950, 0.8470, 0.8712])
model_CB = np.array([0.7953, 0.7582, 0.7670, 0.7599, 0.8607])
model_STA = np.array([0.8459, 0.7856, 0.8470, 0.8377, 0.9024])

# Gabungkan data dalam bentuk array (baris = fold, kolom = model)
data = np.array([model_RF, model_ET, model_ADA, model_GB, model_HGB, model_XGB, model_CB, model_STA]).T

# Friedman Test
stat, p_value = friedmanchisquare(*data)

print(f"Friedman Test Statistic: {stat}, p-value: {p_value}")


Friedman Test Statistic: 26.23899371069184, p-value: 2.8320057539188962e-05


Nemenyi Test

In [62]:
pip install scikit-posthocs

In [73]:
import numpy as np
import scikit_posthocs as sp
import pandas as pd
from scipy.stats import rankdata

# Data hasil K-Fold Cross Validation
# Data hasil K-Fold
model_RF = np.array([0.8206, 0.7823, 0.7982, 0.7840, 0.8640])
model_ET = np.array([0.7628, 0.7352, 0.7982, 0.7664, 0.8520])
model_ADA = np.array([0.8120, 0.7599, 0.7703, 0.8120, 0.8624])
model_GB = np.array([0.8080, 0.7094, 0.7599, 0.7664, 0.8607])
model_HGB = np.array([0.7674, 0.7094, 0.7933, 0.7664, 0.8799])
model_XGB = np.array([0.7887, 0.7094, 0.7950, 0.8470, 0.8712])
model_CB = np.array([0.7953, 0.7582, 0.7670, 0.7599, 0.8607])
model_STA = np.array([0.8459, 0.7856, 0.8470, 0.8377, 0.9024])


# Gabungkan data dalam bentuk array (baris = fold, kolom = model)
data = np.array([model_RF, model_ET, model_ADA, model_GB, model_HGB, model_XGB, model_CB, model_STA]).T

# Buat dataframe dengan nama model
model_names = ["RF", "ET", "ADA", "GB", "HGB", "XGB", "CB", "STA"]
df = pd.DataFrame(data, columns=model_names)

# Nemenyi post-hoc test
nemenyi_results = sp.posthoc_nemenyi_friedman(df.values)

# Tambahkan label model ke hasil
nemenyi_results.columns = model_names
nemenyi_results.index = model_names

print("=== Nemenyi Post-hoc Test Results ===")
print(nemenyi_results)

=== Nemenyi Post-hoc Test Results ===
           RF        ET       ADA        GB       HGB       XGB        CB  \
RF   1.000000  0.525520  0.996738  0.354807  0.742183  0.990868  0.437482   
ET   0.525520  1.000000  0.924296  0.999996  0.999983  0.957576  1.000000   
ADA  0.996738  0.924296  1.000000  0.815971  0.985810  1.000000  0.877180   
GB   0.354807  0.999996  0.815971  1.000000  0.999099  0.877180  1.000000   
HGB  0.742183  0.999983  0.985810  0.999099  1.000000  0.994392  0.999829   
XGB  0.990868  0.957576  1.000000  0.877180  0.994392  1.000000  0.924296   
CB   0.437482  1.000000  0.877180  1.000000  0.999829  0.924296  1.000000   
STA  0.957576  0.049568  0.615019  0.022251  0.119104  0.525520  0.033568   

          STA  
RF   0.957576  
ET   0.049568  
ADA  0.615019  
GB   0.022251  
HGB  0.119104  
XGB  0.525520  
CB   0.033568  
STA  1.000000  
